In [1]:
import numpy as np
import pandas as pd
import logging
import json
import sys
import selenium
import urllib.parse as urlparse
from operator import itemgetter
import time
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from flask import Flask, render_template, request, redirect, url_for, abort
import requests
import copy
import config
import datetime
from IPython.display import display, HTML


# Create the Flask application.
app = Flask(__name__)
app.logger.setLevel(logging.INFO)
app.config.from_object(config.ProductionConfig)
from selenium.webdriver import Chrome
import time


In [9]:
def call_espn_api(league_id):
    app.logger.info('%s - Calling ESPN API', league_id)
    url = ("https://fantasy.espn.com/apis/v3/games/fba/seasons/2021/segments/0/leagues/{}?&view=mMatchupScore&view=mPendingTransactions&view=mPositionalRatings&view=mRoster&view=mTeam&view=modular)".format(league_id))
    try:
        r = requests.get(url)
    except requests.exceptions.RequestException as ex:
        app.logger.error('%s - Could not make ESPN API call', league_id, ex)
        abort(500)
        sys.exit('Could not make ESPN API call.')
    data = r.json()
    return data

In [10]:
class Player:
    def __init__(self, name):
        self.name = name
        self.fga = 1.0
        self.fgm = 0
        self.fgp = self.fgm / self.fga
        self.fta = 1.0
        self.ftm = 0
        self.ftp = self.ftm / self.fta
        self.pts = 0
        self.ast = 0
        self.rbs = 0
        self.tov = 0
        self.m3s = 0
        self.stl = 0
        self.blk = 0
    def __str__(self):
        return self.name + " pts/reb/ast: " + format(self.pts ,'.2f') + "/" + format(self.rbs ,'.2f') + "/" + format(self.ast ,'.2f') + " stl/blk/tov: " + format(self.stl ,'.2f') + "/" + format(self.blk ,'.2f') + "/" + format(self.tov ,'.2f') + "/" + " fg%/ft%/3pm: " + format(self.fgp ,'.2f') + "/" + format(self.ftp ,'.2f') + "/" + format(self.m3s ,'.2f')

In [105]:
def parse_api_to_rosters(hashtag):
    data = call_espn_api(1216230781)
    rosters = [[Player("") for i in range(13)] for j in range(10)]
    teamMap = {}
    for i in range(10):
        teamMap[data["teams"][i]['abbrev']] = i
        for j in range(len(data["teams"][i]['roster']['entries'])):
            rosters[i][j] = Player(data["teams"][i]['roster']['entries'][j]['playerPoolEntry']['player']['fullName'])
            if (hashtag):
                df = pd.read_csv("hashtagprojections.csv")
                df["PLAYER"] = [x.split("\n")[0] for x in df["PLAYER"]]
                df = df.loc[df["PLAYER"] == rosters[i][j].name]
                print(rosters[i][j].name)
                rosters[i][j].pts = df["PTS"].values[0]
                rosters[i][j].rbs = df["TREB"].values[0]
                rosters[i][j].ast = df["AST"].values[0]
                rosters[i][j].stl = df["STL"].values[0]
                rosters[i][j].blk = df["BLK"].values[0]
                rosters[i][j].m3s = df["3PM"].values[0]
                rosters[i][j].tov = df["TO"].values[0]

                rosters[i][j].fgm = df["FG%"].values[0].split("/")[0].split("(")[1]
                rosters[i][j].fga = df["FG%"].values[0].split("/")[1][:-1]

                rosters[i][j].ftm = df["FT%"].values[0].split("/")[0].split("(")[1]
                rosters[i][j].fta = df["FT%"].values[0].split("/")[1][:-1]
            else:
                x = data["teams"][i]['roster']['entries'][j]['playerPoolEntry']['player']['stats']
                print("\n")
                statIndex = 0
                for k in range(len(x)):
                    if x[k]['seasonId'] == 2021 and x[k]['statSplitTypeId'] == 0 and x[k]['externalId'] == '2021' and x[k]['statSourceId'] == 1:
                        statIndex = k
                        print(x[statIndex]['averageStats'])
    #                     if not x[k]['stats']:
    #                         for l in range(len(x)):
    #                             if x[l]['seasonId'] == 2020 and x[l]['statSplitTypeId'] == 0 and x[l]['externalId'] == '2020' and x[l]['statSourceId'] == 0:
    #                                 statIndex = l
                seasonStats = x[statIndex]['averageStats']
                rosters[i][j].pts = seasonStats['0']
                rosters[i][j].fgm = seasonStats['13']
                rosters[i][j].fga = seasonStats['14']
                rosters[i][j].fgp = seasonStats['13'] / seasonStats['14']
                rosters[i][j].ftm = seasonStats['15']
                rosters[i][j].fta = seasonStats['16']
                rosters[i][j].ftp = seasonStats['15'] / seasonStats['16']
                rosters[i][j].rbs = seasonStats['6']
                rosters[i][j].ast = seasonStats['3']
                rosters[i][j].stl = seasonStats['2']
                rosters[i][j].blk = seasonStats['1']
                rosters[i][j].tov = seasonStats['11']
                if '17' not in seasonStats:
                    rosters[i][j].m3s = 0
                else:
                    rosters[i][j].m3s = seasonStats['17']
                print(rosters[i][j])
            
    return rosters, teamMap

In [106]:
def addTeamData(rosters, category):
    team_data = [0 for j in range(10)]
    for i in range(0, 10):
        val = 0
        for j in range(0, len(rosters[i])):
            val += getattr(rosters[i][j], category)
        team_data[i] = val
    return team_data
def rank_list(input, order):
    output = [0] * len(input)
    for i, x in enumerate(sorted(range(len(input)), key=lambda y: order *input[y])):
        output[x] = i + 1
    return output
def printRow(df, index):
    df = {x: y for x, y in zip(df.columns, df.loc[index])}
    sf = pd.DataFrame(df, index=[index])
    display(sf)
def trade_analyzer(rosters, team1, players_from1, team2, players_from2, teamMap):
    if len(players_from1) != len(players_from2):
        print("The number of players exchanged must be equal")
    for player in players_from1:
        [print(p) for p in rosters[teamMap[team1]] if p.name == player]
    for player in players_from2:
        [print(p) for p in rosters[teamMap[team2]] if p.name == player]
    old_data = genRankTable(rosters,teamMap)
    old_rankList, matchups = genRankList(old_data)
    print("Prior to the trade " + team1 + " was ranked "+ str(old_rankList[teamMap[team1]]))
    printRow(old_data, team1)
    print("Prior to the trade " + team2 + " was ranked "+ str(old_rankList[teamMap[team2]]))
    printRow(old_data, team2)
    new_team1 = [player for player in rosters[teamMap[team1]] if player.name not in players_from1]
    new_team2 = [player for player in rosters[teamMap[team2]] if player.name not in players_from2]
    new_rosters =[[j for j in i] for i in rosters]
    new_rosters[team_map[team1]] = new_team1
    new_rosters[team_map[team2]] = new_team2
    new_data = genRankTable(new_rosters, name_mapping, hashtagTable, bbrTable, team_map.keys())
    new_rankList, matchups = genRankList(new_data)
    print("After the trade " + team1 + " was ranked "+ str(new_rankList[team_map[team1]]))
    printRow(new_data, team1)
    print("After the trade " + team2 + " was ranked "+ str(new_rankList[team_map[team2]]))
    printRow(new_data, team2)
def calculate_matchups(team_name, rank_table):
    my_team = rank_table.loc[team_name]
    would_beat = []
    would_lose = []
    overall_margin = []
    for i in range(10):
        margin = []
        score = 0
        curr_team = rank_table.iloc[i]
        if curr_team.name == my_team.name:
            continue
        for j in range(9):
            if my_team.iloc[j] < curr_team.iloc[j]:
                score += 1
                margin.append(curr_team.iloc[j] - my_team.iloc[j])
        if score >= 5:
            would_beat.append(curr_team.name)
        else:
            would_lose.append(curr_team.name)
        avg = sum(margin)/len(margin)
#         print(my_team.name + " " + str(score) + " " + curr_team.name + " " + str(9 - score) + " by an average of " + str(avg))
    return would_beat, would_lose
def genRankList(table):
    num_beaten = [0 for i in range(0, table.shape[0])]
    teams = table.index
    matchups = [[0 for i in range(2)]for i in range(0, table.shape[0])]
    for i in range(len(teams)):
        beats, loses = calculate_matchups(teams[i],table)
        num_beaten[i] = len(beats)
        beats_str = " "
        for j in range(len(beats)):
            beats_str += beats[j] + ", "
        loses_str = " "
        for j in range(len(loses)):
            loses_str += loses[j] + ", "
#         print(teams[i] + " beats" + beats_str + "and loses to" + loses_str)
#         print("")
        matchups[i][0] = beats
        matchups[i][1] = loses
    return rank_list(num_beaten, -1), matchups
def genRankTable(rosters, teamMap):
    fga = addTeamData(rosters, "fga")
    fgm = addTeamData(rosters, "fgm")
    fg = [rank_list([fgm[i]/fga[i] for i in range(0, len(fga))], -1)]
    fta = addTeamData(rosters, "fta")
    ftm = addTeamData(rosters, "ftm")
    ft = [rank_list([ftm[i]/fta[i] for i in range(0, len(fta))], -1)]
    points = [rank_list(addTeamData(rosters, "pts"), -1)]
    rebounds = [rank_list(addTeamData(rosters, "rbs"), -1)]
    assists = [rank_list(addTeamData(rosters, "ast"), -1)]
    steals = [rank_list(addTeamData(rosters, "stl"), -1)]
    blocks = [rank_list(addTeamData(rosters, "blk"), -1)]
    turnovers = [rank_list(addTeamData(rosters, "tov"), 1)]
    threes = [rank_list(addTeamData(rosters, "m3s"), -1)]
    # total = [rankList(addTeamDataHashtag(hashtag_table, 'TOTAL', all_teams, name_mapping), -1)]
    data = [*zip(*(fg + ft + points + rebounds + assists + steals + blocks + turnovers + threes))]
    rank_table = pd.DataFrame(data, index = teamMap.keys(), columns = ['fg%', 'ft%', 'points','rebounds','assists','steals','blocks','turnovers','threes'])
    rank_table["Total"] = rank_table.sum(axis = 1, skipna = True)
    return rank_table
def genStatsTable(rosters, teamList):
    fga = addTeamData(rosters, "fga")
    fgm = addTeamData(rosters, "fgm")
    fg = [[fgm[i]/fga[i] for i in range(0, len(fga))]]
    fta = addTeamData(rosters, "fta")
    ftm = addTeamData(rosters, "ftm")
    ft = [[ftm[i]/fta[i] for i in range(0, len(fta))]]
    points = [addTeamData(rosters, "pts")]
    rebounds = [addTeamData(rosters, "rbs")]
    assists = [addTeamData(rosters, "ast")]
    steals = [addTeamData(rosters, "stl")]
    blocks = [addTeamData(rosters, "blk")]
    turnovers = [addTeamData(rosters, "tov")]
    threes = [addTeamData(rosters, "m3s")]
    # total = [rankList(addTeamDataHashtag(hashtag_table, 'TOTAL', all_teams, name_mapping), -1)]
    data = [*zip(*(fg + ft + points + rebounds + assists + steals + blocks + turnovers + threes))]
    stats_table = pd.DataFrame(data, index = teamMap.keys(), columns = ['fg%', 'ft%', 'points','rebounds','assists','steals','blocks','turnovers','threes'])
    return stats_table


In [108]:
rosters, teamMap = parse_api_to_rosters(hashtag=True)
data = genRankTable(rosters, teamMap)
stats = genStatsTable(rosters, teamMap)
rankList, matchups = genRankList(data)
data["rank"] = rankList
display(data)
display(stats)


[2020-12-21 20:39:52,424] INFO in <ipython-input-9-e9b053154e30>: 1216230781 - Calling ESPN API


Trae Young
Ben Simmons
Nikola Vucevic
Domantas Sabonis
John Wall
Lonzo Ball
Michael Porter Jr.
Richaun Holmes
Al Horford
Jarrett Allen
Wendell Carter Jr.
Evan Fournier
Darius Bazley
Ja Morant
Jamal Murray
Jonas Valanciunas
Malcolm Brogdon
Danilo Gallinari
Buddy Hield
Myles Turner
Malik Beasley
OG Anunoby
DeAndre Jordan
Killian Hayes
LeBron James
Jayson Tatum
James Harden
Joel Embiid
Jrue Holiday
Brandon Ingram
De'Aaron Fox
Montrezl Harrell
Julius Randle
Jerami Grant
Lauri Markkanen
Mike Conley
Aaron Gordon
Otto Porter Jr.
Miles Bridges
Anthony Davis
Russell Westbrook
Bam Adebayo
D'Angelo Russell


IndexError: index 0 is out of bounds for axis 0 with size 0

In [7]:
print(rosters)

[[<__main__.Player object at 0x0000022CD280EDC8>, <__main__.Player object at 0x0000022CD2404808>, <__main__.Player object at 0x0000022CD2616C08>, <__main__.Player object at 0x0000022CD2825208>, <__main__.Player object at 0x0000022CD2825F08>, <__main__.Player object at 0x0000022CD28251C8>, <__main__.Player object at 0x0000022CD2825BC8>, <__main__.Player object at 0x0000022CD2825A48>, <__main__.Player object at 0x0000022CD2825C08>, <__main__.Player object at 0x0000022CD2825B88>, <__main__.Player object at 0x0000022CD28257C8>, <__main__.Player object at 0x0000022CD2825C48>, <__main__.Player object at 0x0000022CD2825B48>], [<__main__.Player object at 0x0000022CD28250C8>, <__main__.Player object at 0x0000022CD2825E48>, <__main__.Player object at 0x0000022CD2825108>, <__main__.Player object at 0x0000022CD2825D08>, <__main__.Player object at 0x0000022CD2825E08>, <__main__.Player object at 0x0000022CD2825088>, <__main__.Player object at 0x0000022CD2819648>, <__main__.Player object at 0x0000022